In [1]:
import _threshold_utils as threshold_utils
import altair as alt

import pandas as pd
import geopandas as gpd
from segment_speed_utils.project_vars import analysis_date
from shared_utils import calitp_color_palette as cp

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/rt_segment_speeds/_threshold_utils.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lat

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/"

In [4]:
analysis_date

'2023-03-15'

In [5]:
speed_stops_subset = ['gtfs_dataset_key', '_gtfs_dataset_name', 'shape_array_key',
       'stop_sequence', 'trip_id','speed_mph']

In [6]:
speed_stops = pd.read_parquet(f"{GCS_PATH}speeds_stop_segments_{analysis_date}")

In [7]:
speed_stops.sample()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,min_time,min_dist,max_time,max_dist,meters_elapsed,sec_elapsed,speed_mph
__null_dask_index__,,,,,,,,,,,,
23727,03cadbffef6f3cd4ce839b218bc65e82,Bay Area 511 Santa Clara Transit VehiclePositions,c680faf3107a29fd9393be3414fb36db,17,3286643,2023-03-15 06:34:12,39.14,2023-03-15 06:37:15,2072.45,2033.31,183.00,24.86


In [8]:
speed_stops2 = speed_stops[speed_stops_subset]

In [9]:
speed_stops2.sample()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,speed_mph
__null_dask_index__,,,,,,
1234394,957ccc3961ce0111ca70c5af9b52ccf8,Fresno Vehicle Positions,e731df74eb8e4457b6fa642e04e532e3,67,558550,NaN


In [10]:
# Do I need geometry? Doesn't seem like it..
avg_speeds = gpd.read_parquet(f"{GCS_PATH}avg_speeds_stop_segments_{analysis_date}.parquet")

In [11]:
avg_speeds_subset = ['geometry','geometry_arrowized','district','district_name']

In [12]:
avg_speeds2 = avg_speeds.drop(columns = avg_speeds_subset)

In [13]:
avg_speeds2.head(2)

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph
0,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00
1,00093e1c28352239174c92c4f07a483b,2,03cadbffef6f3cd4ce839b218bc65e82,0,0.63,26,0.22,0.69


In [14]:
m1 = pd.merge(avg_speeds2, speed_stops2, on = ['gtfs_dataset_key','shape_array_key', 'stop_sequence'], how = 'inner')

In [21]:
avg_speeds2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155454 entries, 0 to 144863
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   shape_array_key   155454 non-null  object 
 1   stop_sequence     155454 non-null  int16  
 2   gtfs_dataset_key  155454 non-null  object 
 3   loop_or_inlining  155454 non-null  int64  
 4   avg_speed_mph     155454 non-null  float64
 5   n_trips           155454 non-null  int64  
 6   p20_speed_mph     155454 non-null  float64
 7   p80_speed_mph     155454 non-null  float64
dtypes: float64(3), int16(1), int64(2), object(2)
memory usage: 9.8+ MB


In [24]:
m1.head()

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph,_gtfs_dataset_name,trip_id,speed_mph
0,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288092,0.00
1,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288102,0.00
2,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288115,0.00
3,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288117,0.00
4,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288099,0.00


In [19]:
m1.columns

Index(['shape_array_key', 'stop_sequence', 'gtfs_dataset_key',
       'loop_or_inlining', 'avg_speed_mph', 'n_trips', 'p20_speed_mph',
       'p80_speed_mph', '_gtfs_dataset_name', 'trip_id', 'speed_mph'],
      dtype='object')

In [26]:
m1.groupby(['_gtfs_dataset_name','shape_array_key']).agg({'n_trips':'max'}).head()

n_trips
_gtfs_dataset_name              shape_array_key                          
Anaheim Resort VehiclePositions 11d91cab41cde51a6d4f623b9cba867c        4
                                1e48661d195c13fc91d1a2f96f1f8c4f       26
                                29d2bbdbeaec1d6888800f85bebf6e33        8
                                2d6ca8831b70c1ff44dc5aa81d9114d0        5
                                39fc40d2ccadfa46548cbf953879ff90        4

In [27]:
test1 = m1.melt(id_vars=[ '_gtfs_dataset_name','shape_array_key','trip_id', 'stop_sequence','gtfs_dataset_key','loop_or_inlining',
                       'n_trips'], value_vars=[ 'avg_speed_mph','speed_mph','p20_speed_mph', 'p80_speed_mph'])

In [36]:
test1.shape

(11174816, 9)

In [39]:
# test1[test1.shape_array_key == "29d2bbdbeaec1d6888800f85bebf6e33"]

In [86]:
# Only need average speed/p20 speed/p80 to show up once for each stop sequence-operator-shape array
test2 = test1.drop_duplicates(subset = [ '_gtfs_dataset_name','shape_array_key','stop_sequence','gtfs_dataset_key','variable','value']).reset_index(drop = True)

In [87]:
test2.shape

(2842750, 9)

### Test with Anaheim First
* Create new col that rounds up speed for plotting purposes only.

In [88]:
anaheim_test = test2[test2.shape_array_key == "29d2bbdbeaec1d6888800f85bebf6e33"]

In [89]:
anaheim_test = anaheim_test.rename(columns = {'value':'speed'})

In [154]:
anaheim_test['rounded_speed'] = anaheim_test.speed.astype(int)

In [155]:
anaheim_test.head()

,_gtfs_dataset_name,shape_array_key,trip_id,stop_sequence,gtfs_dataset_key,loop_or_inlining,n_trips,variable,speed,rounded_speed
144753,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:4,1,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,0.00,0
144754,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:4,2,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,7.99,7
144755,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:8,3,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,4.21,4
144756,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:3,4,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,4.81,4
144757,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:5,5,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,0.61,0


In [156]:
anaheim_test.variable.unique()

array(['avg_speed_mph', 'speed_mph', 'p20_speed_mph', 'p80_speed_mph'],
      dtype=object)

In [157]:
anaheim_test_speedmph = anaheim_test[anaheim_test.variable == 'speed_mph'].reset_index(drop = True)

In [158]:
anaheim_test_other= anaheim_test[anaheim_test.variable != 'speed_mph'].reset_index(drop = True)

In [193]:
anaheim_test_other.head()

,_gtfs_dataset_name,shape_array_key,trip_id,stop_sequence,gtfs_dataset_key,loop_or_inlining,n_trips,variable,speed,rounded_speed
0,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:4,1,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,0.00,0
1,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:4,2,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,7.99,7
2,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:8,3,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,4.21,4
3,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:3,4,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,4.81,4
4,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:5,5,1051e067d18a76b0a79789f051d5c22b,1,8,avg_speed_mph,0.61,0


In [194]:
def create_dot_plot1(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 400).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='ascending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [195]:
chart1 = create_dot_plot1(anaheim_test_other, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_other.columns.tolist(),  'Percentile/Average')

In [196]:
chart1 = threshold_utils.chart_size(chart1,650,300)

In [234]:
chart2 = (
        alt.Chart(anaheim_test, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [235]:
chart2 = threshold_utils.chart_size(chart2,80,300)

In [236]:
chart2

alt.Chart(...)

In [212]:
chart3 = alt.Chart(anaheim_test_speedmph).mark_circle(size=60).encode(
    x='stop_sequence',
    y='rounded_speed',
    color='variable',
    tooltip=anaheim_test_speedmph.columns.tolist()
).interactive()

In [213]:
chart3 = threshold_utils.chart_size(chart3,650,300)

In [214]:
chart3

alt.Chart(...)

In [228]:
chart6 = alt.Chart(anaheim_test_other).mark_circle(size=200).encode(
    x='stop_sequence',
    y='rounded_speed',
    color='variable',
    tooltip= anaheim_test_other.columns.tolist()
).interactive()

In [229]:
chart6 = threshold_utils.chart_size(chart6,650,300)

In [230]:
chart3 + chart6

alt.LayerChart(...)

In [216]:
def create_dot_plot2(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 100).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='ascending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [217]:
chart4 = create_dot_plot2(anaheim_test_speedmph, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_speedmph.columns.tolist(), 'Speed per Trip')

In [218]:
chart4 = threshold_utils.chart_size(chart4,650,300)

In [219]:
chart4

alt.Chart(...)

In [220]:
chart1

alt.Chart(...)

In [222]:
chart1 + chart4

alt.LayerChart(...)

In [223]:
def alt_dropdown(df, col_for_dropdown:str, dropdown_menu_title:str):
    # Create dropdown menu
    # Exclude "none" operators which are only scheduled data
    df = df.loc[df[col_for_dropdown] != "None"][[col_for_dropdown]]
    dropdown_list = df[col_for_dropdown].unique().tolist()
    
    # Show only first operator by default
    initialize_first_op = sorted(dropdown_list)[0]
    input_dropdown = alt.binding_select(options=sorted(dropdown_list), name=dropdown_menu_title)
    
    selection = alt.selection_single(name= dropdown_menu_title,fields=[col_for_dropdown],
    bind=input_dropdown, init={col_for_dropdown: initialize_first_op})
                 
    return selection